In [1]:
import requests
import pandas as pd
from pyspark.sql import SparkSession
from bs4 import BeautifulSoup, NavigableString, Tag

URL = "https://thepurplepigchicago.com/drink"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
menu = soup.find_all("div", class_="menu-section")

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [20]:
def get_wines_df():
    sections = []
    items = []
    staging = []
    def clean_extra(extra):
        if extra == None:
            return ''
        else:
            return ' '.join(x.strip() for x in extra.replace('\n','').split('/'))

    for i in menu:
        sections.append((i.find('div',{'class':'menu-section-title'}).text, \
                             zip([x.text.split('|')[1] if '|' in x.text else x.text for x in i.find_all('div',{'class':'menu-item-title'})], \
                            [x.text.replace('|','') for x in i.find_all("div", class_="menu-item-description") if x != None], \
                            [clean_extra(x.text) for x in i.find_all("div", class_="menu-item-price-bottom")])))



    for i in sections:
        category = i[0]
        for items in i[1]:
            staging.append(["PurplePig"]+[category] + [x for x in items])

    dict_holder = []
    key_list = ['origin','section', 'name', 'description','extra']

    for i in staging:
        dict_from_list = dict(zip(key_list, i))
        dict_holder.append(dict_from_list)
    return(pd.DataFrame(dict_holder).apply(lambda x: x.str.strip() if x.dtype == "object" else x))
    

In [21]:
df = get_wines_df()

In [5]:
df.to_parquet('Wines.parquet')

In [6]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [7]:
df = spark.read.parquet('Wines.parquet')

In [8]:
df.write.format('delta').save('tmp/delta/')

In [10]:
spark.sql("CREATE TABLE wines USING DELTA LOCATION './tmp/delta'")

DataFrame[]

In [27]:
spark.sql("SELECT * FROM wines WHERE name like '%BLANC%' LIMIT 10;").show()

+---------+--------------------+----------------+--------------------+-----------+
|   origin|             section|            name|         description|      extra|
+---------+--------------------+----------------+--------------------+-----------+
|PurplePig|SOMMELIER WINE SP...| SAUVIGNON BLANC|Patrick Noël, San...|$19 $45 $86|
|PurplePig| WHITES BY THE GLASS| SAUVIGNON BLANC|Chateau La Rame, ...|$15 $36 $68|
+---------+--------------------+----------------+--------------------+-----------+



In [28]:
OUTPUT_DELTA_PATH = './tmp/delta/'

spark.sql('CREATE DATABASE IF NOT EXISTS WINES')

spark.sql('''
    CREATE TABLE IF NOT EXISTS WINES.PURPLE_PIG(
        origin string
        , section string
        , name string
        , description string
        , extra string
    ) USING DELTA
    LOCATION "{0}"
    '''.format(OUTPUT_DELTA_PATH)
)

DataFrame[]

In [29]:
load_file = spark.read.parquet('Wines.parquet')

In [35]:
load_file.createOrReplaceTempView('wine_load')

In [34]:
spark.sql("""SELECT count(1)
FROM WINES.PURPLE_PIG""").show()

+--------+
|count(1)|
+--------+
|     544|
+--------+



In [43]:
spark.sql("""SELECT hash(origin,section, name, description)
FROM wine_load""").show()

+----------------------------------------+
|hash(origin, section, name, description)|
+----------------------------------------+
|                              2061198233|
|                              1527710402|
|                              -632903228|
|                              1261612108|
|                                 2099280|
|                             -2124324733|
|                             -1568785439|
|                             -1899088557|
|                             -1618643588|
|                             -1993399701|
|                              -515037650|
|                               -21673382|
|                             -1172576607|
|                               128985245|
|                             -1285102648|
|                              1795557171|
|                              1636546046|
|                               643601468|
|                             -1081905292|
|                              -444497963|
+----------

In [44]:
spark.sql("""MERGE INTO WINES.PURPLE_PIG
USING wine_load
   ON  hash(WINES.PURPLE_PIG.origin,WINES.PURPLE_PIG.section, WINES.PURPLE_PIG.name, WINES.PURPLE_PIG.description) = \
    hash(wine_load.origin,wine_load.section, wine_load.name, wine_load.description)
 WHEN NOT MATCHED THEN
 	  INSERT (origin, section, name, description) VALUES (origin, section, name, description)
""")

DataFrame[]